In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!cp "/content/drive/My Drive/colab_drive/Dataset.zip" /content/

In [0]:
import os
os.chdir("/content/")
!unzip Dataset.zip -d /content/

Archive:  Dataset.zip
replace /content/Dataset/Testing_Dataset/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
from keras import applications
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [0]:
!ls
os.chdir("Dataset")
y_train = list(os.listdir("Training_Dataset"))
y_test = list(os.listdir("Testing_Dataset"))
print(y_test)
print(y_train)

Dataset  Dataset.zip  drive  sample_data
['Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Peach___Bacterial_spot', 'Peach___healthy']
['Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Peach___Bacterial_spot', 'Peach___healthy']


In [0]:
train_datagen_VGG16 = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator_VGG16 = train_datagen_VGG16.flow_from_directory(
    "Training_Dataset",
    target_size=(224,224),
    batch_size=64,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator_VGG16 = train_datagen_VGG16.flow_from_directory(
    "Testing_Dataset", # same directory as training data
    target_size=(224,224),
    batch_size=64,
    class_mode="categorical",
    subset='validation') # set as validation data


Found 3537 images belonging to 5 classes.
Found 218 images belonging to 5 classes.


In [0]:
model = applications.VGG16(weights = "imagenet", include_top=False,
                           input_shape = (224,224, 3))

In [0]:
# Freeze the layers which we don't want to train. Here I am freezing all layers.
for layer in model.layers[0:-1]:
    layer.trainable = False    

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.2)(x)

predictions = Dense(5, activation="softmax")(x)

In [0]:
# creating the final model 
model_final = Model(input = model.input, output = predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [0]:
model_final.summary()

In [0]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001), metrics=["accuracy"])

In [0]:

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [0]:
#!cp "/content/drive/My Drive/colab_drive/vgg16_1.h5" "/content/Dataset/vgg16_1.h5"
#model_final.load_weights("/content/vgg16_1.h5")

In [0]:
model_final.fit_generator(
    train_generator_VGG16,
    steps_per_epoch = train_generator_VGG16.samples // 64,
    validation_data = validation_generator_VGG16, 
    validation_steps = validation_generator_VGG16.samples // 64,
    epochs = 3,
    callbacks = [checkpoint, early])

Epoch 1/3
55/55 [==============================] - 46s 838ms/step - loss: 2.5018 - acc: 0.7591 - val_loss: 1.6282 - val_acc: 0.8802

Epoch 00001: val_acc improved from -inf to 0.88021, saving model to vgg16_1.h5
Epoch 2/3
55/55 [==============================] - 39s 715ms/step - loss: 1.1259 - acc: 0.8765 - val_loss: 0.6573 - val_acc: 0.9351

Epoch 00002: val_acc improved from 0.88021 to 0.93506, saving model to vgg16_1.h5
Epoch 3/3
55/55 [==============================] - 40s 722ms/step - loss: 0.6301 - acc: 0.9201 - val_loss: 0.4291 - val_acc: 0.9481

Epoch 00003: val_acc improved from 0.93506 to 0.94805, saving model to vgg16_1.h5
